## Introduction

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Overview of the Data

The following will display a preview of the data so we can get a feel for it.

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
    FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* `name` — the name of the country.
* `area`— the country's total area (both land and water).
* `area_land` — the country's land area in square kilometers.
* `area_water` — the country's waterarea in square kilometers.
* `population` — the country's population.
* `population_growth`— the country's population growth as a percentage.
* `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
* `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

Let's start by calculating some summary statistics and look for any outlier countries.

In [5]:
%%sql

SELECT MIN(population) AS 'Minimum Population', MAX(population) AS 'Maximum Population', 
MIN(population_growth) AS 'Minimum Population Growth', MAX(population_growth) AS 'Maximum Population Growth'
    FROM facts;

Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics on the previous screen:

There's a country with a population of 0
There's a country with a population of 7256490011 (or more than 7.2 billion people)
Let's use subqueries to zoom in on just these countries without using the specific values.

## Exploring Outliers

In [6]:
%%sql

SELECT * 
    FROM facts
 WHERE population == (SELECT MIN(population) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [7]:
%%sql

SELECT * 
    FROM facts
 WHERE population == (SELECT Max(population) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


At first glance, one could guess that the 7.2 billion population was referring to the world. In regards to the populaton of 0, this is the continent of Anarctica.

In [10]:
%%sql

SELECT MIN(population) AS 'Minimum Population', 
MAX(population) AS 'Maximum Population', 
MIN(population_growth) AS 'Minimum Population Growth', 
MAX(population_growth) AS 'Maximum Population Growth'
    FROM facts
    WHERE name <> 'World';

Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,1367485388,0.0,4.02


In [14]:
%%sql
SELECT AVG(population) as Avg_pop, AVG(area) as Avg_area
    FROM facts
    WHERE name <> 'World';

Done.


Avg_pop,Avg_area
32242666.56846473,555093.546184739


In [17]:
%%sql

SELECT *
    FROM facts
    WHERE population > (SELECT AVG(population) FROM facts WHERE name <> 'World') AND 
    area < (SELECT AVG(area) FROM facts WHERE name <> 'World');


Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


In [21]:
%%sql

SELECT name, MAX(population) as Highest_Population
    FROM facts
    WHERE name <> 'World';  

Done.


name,Highest_Population
China,1367485388


In [22]:
%%sql

SELECT name, MAX(population_growth) as Highest_Population_Growth
    FROM facts
    WHERE name <> 'World';

Done.


name,Highest_Population_Growth
South Sudan,4.02


In [31]:
%%sql

SELECT name, ROUND(CAST(area_water as Float) / CAST(area_land as Float), 2) as water_land_ratio
    FROM facts
    WHERE name <> 'World'
    ORDER BY water_land_ratio DESC;

Done.


name,water_land_ratio
British Indian Ocean Territory,905.67
Virgin Islands,4.52
Puerto Rico,0.55
"Bahamas, The",0.39
Guinea-Bissau,0.28
Malawi,0.26
Netherlands,0.23
Uganda,0.22
Eritrea,0.16
Liberia,0.16


In [32]:
%%sql

SELECT name, ROUND(CAST(area_land as Float) / CAST(area_water as Float), 2) as 
land_to_water_ratio
    FROM facts
    WHERE name <> 'World'
    ORDER BY land_to_water_ratio DESC;

Done.


name,land_to_water_ratio
Bosnia and Herzegovina,5118.7
Niger,4222.33
Morocco,1785.2
Guinea,1755.12
Costa Rica,1276.5
Djibouti,1159.0
"Korea, North",926.22
Cyprus,924.1
Namibia,821.65
Burkina Faso,684.5
